In [2]:
#Importing the needed libraries
import yfinance as yf
import numpy as np
import pandas as pd

In [52]:
#Using yfinance instead of pandas.datareader as it has problems now with yahoo API
tickers = ['AAPL', '^GSPC'] #we use the ticker in the brackets, thus we need '^GSPC' for S&P 500
multpl_stocks = yf.download(tickers,
  start = '2007-11-01',
  end = '2009-03-01')
multpl_stocks = multpl_stocks[['Adj Close']] #here you specify the price type: open, close
multpl_stocks

[*********************100%***********************]  2 of 2 completed


Adj Close             
                AAPL        ^GSPC
Date                             
2007-11-01  5.697629  1508.439941
2007-11-02  5.710699  1509.650024
2007-11-05  5.659329  1502.170044
2007-11-06  5.829858  1520.270020
2007-11-07  5.662977  1475.619995
...              ...          ...
2009-02-23  2.643027   743.330017
2009-02-24  2.743337   773.140015
2009-02-25  2.770998   764.900024
2009-02-26  2.711116   752.830017
2009-02-27  2.714764   735.090027

[333 rows x 2 columns]

In [53]:
#sorting values in the ascending order
multpl_stocks.sort_values(by='Date')
multpl_stocks

Adj Close             
                AAPL        ^GSPC
Date                             
2007-11-01  5.697629  1508.439941
2007-11-02  5.710699  1509.650024
2007-11-05  5.659329  1502.170044
2007-11-06  5.829858  1520.270020
2007-11-07  5.662977  1475.619995
...              ...          ...
2009-02-23  2.643027   743.330017
2009-02-24  2.743337   773.140015
2009-02-25  2.770998   764.900024
2009-02-26  2.711116   752.830017
2009-02-27  2.714764   735.090027

[333 rows x 2 columns]

In [5]:
#Finding returns and deleting NANs
returns = np.log(multpl_stocks).diff()
returns= returns.dropna()
returns

Adj Close          
                AAPL     ^GSPC
Date                          
2007-11-02  0.002291  0.000802
2007-11-05 -0.009036 -0.004967
2007-11-06  0.029687  0.011977
2007-11-07 -0.029043 -0.029810
2007-11-08 -0.059890 -0.000576
...              ...       ...
2009-02-23 -0.047722 -0.035315
2009-02-24  0.037250  0.039320
2009-02-25  0.010033 -0.010715
2009-02-26 -0.021847 -0.015906
2009-02-27  0.001345 -0.023846

[332 rows x 2 columns]

In [42]:
#showing the event date 
returns.loc['2009-01-05']

Adj Close  AAPL     0.041338
           ^GSPC   -0.004679
Name: 2009-01-05 00:00:00, dtype: float64

In [11]:
#accessing 10 days before and after the event date
event = returns.loc['2008-12-18':'2009-01-20']
event

Adj Close          
                AAPL     ^GSPC
Date                          
2008-12-18  0.003024 -0.021390
2008-12-19  0.006353  0.002933
2008-12-22 -0.048490 -0.018472
2008-12-23  0.007436 -0.009765
2008-12-24 -0.015634  0.005764
2008-12-26  0.009014  0.005342
2008-12-29  0.009279 -0.003880
2008-12-30 -0.003701  0.024114
2008-12-31 -0.010954  0.014059
2009-01-02  0.061348  0.031119
2009-01-05  0.041338 -0.004679
2009-01-06 -0.016632  0.007787
2009-01-07 -0.021845 -0.030469
2009-01-08  0.018399  0.003391
2009-01-09 -0.023135 -0.021533
2009-01-12 -0.021425 -0.022823
2009-01-13 -0.010773  0.001757
2009-01-14 -0.027510 -0.034032
2009-01-15 -0.023118  0.001328
2009-01-16 -0.012673  0.007533
2009-01-20 -0.051466 -0.054262

In [7]:
#showing other 252 trading days
tdata = returns.loc['2007-12-19':'2008-12-17']
df = tdata
df

Adj Close          
                AAPL     ^GSPC
Date                          
2007-12-19  0.000765 -0.001362
2007-12-20  0.022089  0.004888
2007-12-21  0.035163  0.016532
2007-12-24  0.024905  0.008045
2007-12-26  0.000754  0.000808
...              ...       ...
2008-12-11 -0.033231 -0.028939
2008-12-12  0.033842  0.007004
2008-12-15 -0.036477 -0.012767
2008-12-16  0.007151  0.050085
2008-12-17 -0.067960 -0.009639

[252 rows x 2 columns]

In [38]:
#regression model for analysis of 252 days
import statsmodels.api as sm
y = tdata.iloc[:, 0] #y is Apple return
X = tdata.iloc[:, 1] #x is S&P500 return

# Adding a constant
X = sm.add_constant(X)
model = sm.OLS(y, X).fit()

# Getting the intercept and slope coefficients
intercept = model.params[0]
slope = model.params[1]

# Calculating R, SE and printing results
R_squared = model.rsquared
SE = np.sqrt(model.mse_resid) #SE as a square root of a mean square error
print('intercept', intercept, 'slope', slope, 'R_squared:', R_squared, 'SE:', SE)

intercept -0.0010022150515650826 slope 0.9809284389191179 R_squared: 0.466002681111393 SE: 0.027152543777706095


In [35]:
print(model.summary())

                              OLS Regression Results                             
Dep. Variable:     ('Adj Close', 'AAPL')   R-squared:                       0.466
Model:                               OLS   Adj. R-squared:                  0.464
Method:                    Least Squares   F-statistic:                     218.2
Date:                   Tue, 14 Mar 2023   Prob (F-statistic):           6.44e-36
Time:                           18:36:22   Log-Likelihood:                 552.22
No. Observations:                    252   AIC:                            -1100.
Df Residuals:                        250   BIC:                            -1093.
Df Model:                              1                                         
Covariance Type:               nonrobust                                         
                             coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------------------------------------------------------

In [43]:
#Accesing the events window,step by step, first we need S&P500 returns for ER
SP = event.iloc[:, 1]
SP

Date
2008-12-18   -0.021390
2008-12-19    0.002933
2008-12-22   -0.018472
2008-12-23   -0.009765
2008-12-24    0.005764
2008-12-26    0.005342
2008-12-29   -0.003880
2008-12-30    0.024114
2008-12-31    0.014059
2009-01-02    0.031119
2009-01-05   -0.004679
2009-01-06    0.007787
2009-01-07   -0.030469
2009-01-08    0.003391
2009-01-09   -0.021533
2009-01-12   -0.022823
2009-01-13    0.001757
2009-01-14   -0.034032
2009-01-15    0.001328
2009-01-16    0.007533
2009-01-20   -0.054262
Name: (Adj Close, ^GSPC), dtype: float64

In [44]:
df1 = pd.DataFrame({'SP': SP})

# Calculating expected returns
df1['Expected_Return'] = intercept + slope * df1['SP']
df1

                  SP  Expected_Return
Date                                 
2008-12-18 -0.021390        -0.021984
2008-12-19  0.002933         0.001874
2008-12-22 -0.018472        -0.019122
2008-12-23 -0.009765        -0.010581
2008-12-24  0.005764         0.004652
2008-12-26  0.005342         0.004238
2008-12-29 -0.003880        -0.004808
2008-12-30  0.024114         0.022652
2008-12-31  0.014059         0.012789
2009-01-02  0.031119         0.029523
2009-01-05 -0.004679        -0.005592
2009-01-06  0.007787         0.006636
2009-01-07 -0.030469        -0.030890
2009-01-08  0.003391         0.002324
2009-01-09 -0.021533        -0.022125
2009-01-12 -0.022823        -0.023390
2009-01-13  0.001757         0.000721
2009-01-14 -0.034032        -0.034386
2009-01-15  0.001328         0.000301
2009-01-16  0.007533         0.006387
2009-01-20 -0.054262        -0.054229


In [45]:
# selecting the first column from the 'event' list 
AAPL_returns = event.iloc[:, 0]

# adding the 'AAPL_returns' to the DataFrame
df1['AAPL_returns'] = AAPL_returns
df1

,SP,Expected_Return,AAPL_returns
Date,,,
2008-12-18,-0.021390,-0.021984,0.003024
2008-12-19,0.002933,0.001874,0.006353
2008-12-22,-0.018472,-0.019122,-0.048490
2008-12-23,-0.009765,-0.010581,0.007436
2008-12-24,0.005764,0.004652,-0.015634
2008-12-26,0.005342,0.004238,0.009014
2008-12-29,-0.003880,-0.004808,0.009279
2008-12-30,0.024114,0.022652,-0.003701
2008-12-31,0.014059,0.012789,-0.010954


In [46]:
df1['Abnormal Return'] = df1['AAPL_returns'] - df1['Expected_Return'] #calculating abnormal return
df1['CAR'] = df1['Abnormal Return'].cumsum() #calculating the cumulative AR
df1

,SP,Expected_Return,AAPL_returns,Abnormal Return,CAR
Date,,,,,
2008-12-18,-0.021390,-0.021984,0.003024,0.025008,0.025008
2008-12-19,0.002933,0.001874,0.006353,0.004479,0.029487
2008-12-22,-0.018472,-0.019122,-0.048490,-0.029369,0.000118
2008-12-23,-0.009765,-0.010581,0.007436,0.018017,0.018135
2008-12-24,0.005764,0.004652,-0.015634,-0.020286,-0.002151
2008-12-26,0.005342,0.004238,0.009014,0.004776,0.002625
2008-12-29,-0.003880,-0.004808,0.009279,0.014088,0.016713
2008-12-30,0.024114,0.022652,-0.003701,-0.026353,-0.009641
2008-12-31,0.014059,0.012789,-0.010954,-0.023742,-0.033383


In [47]:
#Calculating AR t-test as Abnormal Return / Standard Error
df1['AR t-test'] = df1['Abnormal Return'] / SE
df1

,SP,Expected_Return,AAPL_returns,Abnormal Return,CAR,AR t-test
Date,,,,,,
2008-12-18,-0.021390,-0.021984,0.003024,0.025008,0.025008,0.921016
2008-12-19,0.002933,0.001874,0.006353,0.004479,0.029487,0.164958
2008-12-22,-0.018472,-0.019122,-0.048490,-0.029369,0.000118,-1.081618
2008-12-23,-0.009765,-0.010581,0.007436,0.018017,0.018135,0.663554
2008-12-24,0.005764,0.004652,-0.015634,-0.020286,-0.002151,-0.747128
2008-12-26,0.005342,0.004238,0.009014,0.004776,0.002625,0.175886
2008-12-29,-0.003880,-0.004808,0.009279,0.014088,0.016713,0.518840
2008-12-30,0.024114,0.022652,-0.003701,-0.026353,-0.009641,-0.970557
2008-12-31,0.014059,0.012789,-0.010954,-0.023742,-0.033383,-0.874406


In [54]:
#Alternatively we can save the downloaded data of returns and prices and conduct the same analysis ine excel
writer = pd.ExcelWriter('Event study.xlsx', engine='xlsxwriter')
multpl_stocks.to_excel(writer, sheet_name='Prices',startrow=0 , startcol=0)
returns.to_excel(writer, sheet_name='Returns',startrow=0 , startcol=0)
writer.save()